### 숙지 사항 : 
XPath(XML Path Language)는 웹 페이지의 HTML 요소를 찾기 위한 언어입니다. 마치 파일 시스템의 경로처럼, HTML 문서의 특정 요소의 위치를 지정할 수 있습니다.

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse
import time
import os  # 경로 처리를 위해 추가

# 드라이버 설정
chrome_options = Options()
# 현재 작업 디렉토리의 절대 경로를 가져옴
current_directory = os.path.abspath(os.getcwd())
file_path = os.path.join(current_directory, '논문목록')
#다운로드 경로 설정
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": file_path, #다운로드 저장 경로
    "download.prompt_for_download": False, # 파일 다운로드 시 "저장" 또는 "열기" 대화상자를 표시하지 않음
    "download.directory_upgrade": True #이전 다운로드 설정을 새로운 설정으로 업그레이드
})
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
wait = WebDriverWait(driver, 10)
search_term = "인공지능"
try:
    for i in range(0, 500, 100):
        # URL 생성
        serviceUrl = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword="
        param = f"&strQuery={urllib.parse.quote(search_term)}" #한글 utf-8인코딩
        param += f"&query={urllib.parse.quote(search_term)}" #한글 utf-8인코딩
        param += f"&iStartCount={i}"
        url = serviceUrl + param
        
        # 페이지 접속
        driver.get(url)
        
        # (1) 전체선택 클릭
        elem = wait.until(
            EC.presence_of_element_located((By.XPATH, 
                '//*[@id="divContent"]/div/div[2]/div/div[3]/div[1]/div[1]/label/span'))
        )
        elem.click()
        
        # (2) 내보내기 클릭
        driver.execute_script("exportData()")
        
        # (3) 새창 핸들링
        # 새 창이 열릴 때까지 잠시 대기
        time.sleep(2)
        # 새 창으로 전환
        driver.switch_to.window(driver.window_handles[1])
        
        # (4) Excel 저장 클릭
        excel_button = wait.until(
            EC.element_to_be_clickable((By.XPATH,
                '//*[@id="wrap"]/form/div/div[2]/div[1]/div/ul/li[3]/label'))
        )
        excel_button.click()
        
        # (5) 내보내기 클릭
        driver.execute_script("f_submit()")
        
        # (6) 현재창 닫기
        driver.close()
        
        # (7) 본래 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
        
        # 다음 반복을 위한 대기
        time.sleep(2)

except Exception as e:
    print(f"오류 발생: {e}")
    
finally:
    driver.quit()